In [ ]:
print("Hello World!")

In [ ]:
%pip install dlib face_recognition

In [ ]:
%pip install opencv-python face_recognition requests
%pip install --upgrade pip


In [ ]:
%pip install transformers torch speechrecognition pyttsx3

In [ ]:
%pip install pyaudio --global-option='build_ext' --global-option='-I/usr/local/include' --global-option='-L/usr/local/lib'


In [ ]:
%pip install SpeechRecognition pyttsx3 pyaudio


In [ ]:
import os
os.environ["FLAC"] = "ffmpeg"
import speech_recognition as sr
import pyttsx3
engine = pyttsx3.init()
engine.say("Hello Chief, how can I help you?")
engine.runAndWait()


r = sr.Recognizer()
with sr.Microphone() as source:
    print("Listening...")
    audio = r.listen(source)
    
    try:
        query = r.recognize_google(audio)
        print("You said:", query)
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")


In [ ]:
import os
import speech_recognition as sr
import pyttsx3
from transformers import pipeline

# Set FLAC environment for speech recognition
os.environ["FLAC"] = "ffmpeg"

# Initialize text-to-speech engine
engine = pyttsx3.init()
engine.say("Hello Chief, how can I help you?")
engine.runAndWait()

# Load the GPT-2 AI chatbot model
chatbot = pipeline("text-generation", model="gpt2")

# Function to get response from GPT model
def ask_question(question):
    result = chatbot(question, max_length=100, num_return_sequences=1)
    return result[0]['generated_text']

# Function for voice recognition
def listen_and_respond():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = r.listen(source)
        
        try:
            query = r.recognize_google(audio)
            print("You said:", query)

            # Ask GPT-based model for response
            response = ask_question(query)
            print("AI:", response)

            # Speak the response
            engine.say(response)
            engine.runAndWait()

        except sr.UnknownValueError:
            print("Sorry, I could not understand the audio.")
            engine.say("Sorry, I could not understand the audio.")
            engine.runAndWait()

        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            engine.say("There was an issue with the speech service.")
            engine.runAndWait()

# Start listening for commands
listen_and_respond()


In [ ]:
import time
import json
import requests
import speech_recognition as sr
import pyttsx3
import cv2
import face_recognition

# Setup for voice engine
engine = pyttsx3.init()

# Function to speak out a message
def speak(message):
    print(f'Speaking: {message}')
    engine.say(message)
    engine.runAndWait()

# Function to detect motion (placeholder for actual motion detection logic)
def detect_motion():
    # Implement your motion detection logic here
    # For demonstration, we will return True to simulate motion detected
    return True

# Function to detect faces
def detect_faces(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    return face_locations

# Function to control light (simulated)
def control_light(action):
    if action == "on":
        speak("Light turned on")
        # Add actual code to turn on the light
    elif action == "off":
        speak("Light turned off")
        # Add actual code to turn off the light

# Function to get sensor data
def get_sensor_data():
    try:
        response = requests.get("http://your-sensor-url")  # Replace with your actual sensor URL
        response.raise_for_status()  # Raises an error for bad responses
        data = response.json()  # Assuming the response is in JSON format
        return data['temperature'], data['humidity']
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None, None

# Main function to run the home automation system
def main():
    speak("System is now active, waiting for motion detection.")

    # Start the webcam to detect faces
    video_capture = cv2.VideoCapture(0)

    while True:
        if detect_motion():
            control_light("on")
            ret, frame = video_capture.read()

            if ret:
                face_locations = detect_faces(frame)
                num_faces = len(face_locations)
                if num_faces > 0:
                    speak(f"{num_faces} face(s) detected.")
                    control_light("off")
                    speak("Hi Chief!")

                    # Get sensor data
                    temperature, humidity = get_sensor_data()
                    if temperature is not None and humidity is not None:
                        speak(f"The temperature is {temperature} degrees and humidity is {humidity}%.")
                    else:
                        speak("Sorry, I couldn't retrieve the temperature and humidity.")
                else:
                    speak("No faces detected.")

            time.sleep(2)  # Adjust the delay as needed

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("Program terminated.")
    finally:
        video_capture.release()  # Release the webcam
        cv2.destroyAllWindows()  # Close all OpenCV windows


In [ ]:
import os
import random
import datetime
import time
import pyttsx3
import speech_recognition as sr
import requests
from packaging.version import Version
from zeroconf import Zeroconf, ServiceBrowser

# Initialize the speech engine
engine = pyttsx3.init()
voices = engine.getProperty("voices")
engine.setProperty("voice", voices[0].id)

TASKS_FILE = "tasks_reminders.txt"
MUSIC_DIR = "/path/to/your/music"  # Update this to the correct music folder path

# Zeroconf setup to find ESP32
class MyListener:
    def __init__(self):
        self.esp32_ip = None

    def add_service(self, zeroconf, type, name):
        info = zeroconf.get_service_info(type, name)
        if info:
            ip = ".".join(map(str, info.addresses[0]))
            if "esp32-device" in name:
                self.esp32_ip = ip

def get_esp32_ip():
    zeroconf = Zeroconf()
    listener = MyListener()
    browser = ServiceBrowser(zeroconf, "_http._tcp.local.", listener)
    time.sleep(5)
    zeroconf.close()
    return listener.esp32_ip

ESP32_IP = f"http://{get_esp32_ip()}" if get_esp32_ip() else None

def speak(audio):
    engine.say(audio)
    engine.runAndWait()
    print(f"Speaking: {audio}")

def greet_user():
    hour = datetime.datetime.now().hour
    if hour < 12:
        speak("Good Morning Chief!")
    elif hour < 18:
        speak("Good Afternoon Chief!")
    else:
        speak("Good Evening Chief!")
    speak("I am Jarvis, your assistant. How can I assist you today?")

def takecommand(timeout=5):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = r.listen(source, timeout=timeout)
    try:
        query = r.recognize_google(audio, language="en-in")
        print(f"You said: {query}")
    except sr.UnknownValueError:
        return "none"
    except sr.RequestError as e:
        print(f"Error: {e}")
        return "none"
    return query.lower()

def control_light(light_number, action):
    if ESP32_IP:
        url = f"{ESP32_IP}/light{light_number}/{action}"
        try:
            response = requests.get(url)
            if response.status_code == 200:
                speak(f"Light {light_number} turned {action}.")
        except requests.RequestException as e:
            speak(f"Error controlling light {light_number}. Error: {e}")
    else:
        speak("ESP32 is not reachable.")

def play_song():
    try:
        songs = os.listdir(MUSIC_DIR)
        song = random.choice(songs)
        speak(f"Playing {song}")
        os.startfile(os.path.join(MUSIC_DIR, song))
    except Exception as e:
        speak(f"Error playing music: {e}")

def set_reminder(reminder_time, reminder_message):
    reminder_time = datetime.datetime.strptime(reminder_time, "%H:%M").replace(
        year=datetime.datetime.now().year, month=datetime.datetime.now().month, day=datetime.datetime.now().day)
    with open(TASKS_FILE, "a") as file:
        file.write(f"{reminder_time} | {reminder_message}\n")
    speak(f"Reminder set for {reminder_time.strftime('%H:%M')}")

def respond(query):
    if "time" in query:
        speak(f"The time is {datetime.datetime.now().strftime('%H:%M:%S')}")
    elif "date" in query:
        speak(f"Today's date is {datetime.datetime.now().strftime('%A, %B %d, %Y')}")
    elif "play" in query and "song" in query:
        play_song()
    elif "light" in query:
        if "on" in query:
            control_light(query.split("light")[1].strip(), "on")
        elif "off" in query:
            control_light(query.split("light")[1].strip(), "off")
    elif "remind" in query:
        if "at" in query:
            time_part = query.split("at")[1].strip()
            reminder_message = query.split("remind me to")[1].split("at")[0].strip()
            set_reminder(time_part, reminder_message)
        else:
            speak("Please specify the time for the reminder.")
    else:
        speak("I didn't understand that.")

def listen_for_jarvis():
    greet_user()
    while True:
        query = takecommand()
        if "jarvis" in query:
            speak("I'm listening, Chief.")
            query = takecommand()
            respond(query)

if __name__ == "__main__":
    listen_for_jarvis()


In [ ]:
import os
import random
import datetime
import time
import pyttsx3
import speech_recognition as sr

# Initialize the speech engine
engine = pyttsx3.init()
voices = engine.getProperty("voices")
engine.setProperty("voice", voices[0].id)

TASKS_FILE = "tasks_reminders.txt"
MUSIC_DIR = "/path/to/your/music"  # Update this to the correct music folder path

def speak(audio):
    engine.say(audio)
    engine.runAndWait()
    print(f"Speaking: {audio}")

def greet_user():
    hour = datetime.datetime.now().hour
    if hour < 12:
        speak("Good Morning Chief!")
    elif hour < 18:
        speak("Good Afternoon Chief!")
    else:
        speak("Good Evening Chief!")
    speak("I am Jarvis, your assistant. How can I assist you today?")

def takecommand(timeout=5):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = r.listen(source, timeout=timeout)
    try:
        query = r.recognize_google(audio, language="en-in")
        print(f"You said: {query}")
    except sr.UnknownValueError:
        return "none"
    except sr.RequestError as e:
        print(f"Error: {e}")
        return "none"
    return query.lower()

def play_song():
    try:
        songs = os.listdir(MUSIC_DIR)
        song = random.choice(songs)
        speak(f"Playing {song}")
        os.startfile(os.path.join(MUSIC_DIR, song))
    except Exception as e:
        speak(f"Error playing music: {e}")

def set_reminder(reminder_time, reminder_message):
    reminder_time = datetime.datetime.strptime(reminder_time, "%H:%M").replace(
        year=datetime.datetime.now().year, month=datetime.datetime.now().month, day=datetime.datetime.now().day)
    with open(TASKS_FILE, "a") as file:
        file.write(f"{reminder_time} | {reminder_message}\n")
    speak(f"Reminder set for {reminder_time.strftime('%H:%M')}")

def respond(query):
    if "time" in query:
        speak(f"The time is {datetime.datetime.now().strftime('%H:%M:%S')}")
    elif "date" in query:
        speak(f"Today's date is {datetime.datetime.now().strftime('%A, %B %d, %Y')}")
    elif "play" in query and "song" in query:
        play_song()
    elif "remind" in query:
        if "at" in query:
            time_part = query.split("at")[1].strip()
            reminder_message = query.split("remind me to")[1].split("at")[0].strip()
            set_reminder(time_part, reminder_message)
        else:
            speak("Please specify the time for the reminder.")
    else:
        speak("I didn't understand that.")

def listen_for_jarvis():
    greet_user()
    while True:
        query = takecommand()
        if "jarvis" in query:
            speak("I'm listening, Chief.")
            query = takecommand()
            respond(query)

if __name__ == "__main__":
    listen_for_jarvis()


In [15]:
for i in range(700):
    print("i love u Teju",end=" ")

i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love u Teju i love